In [ ]:
import numpy as np

#fungsi distribusi konsentrasi C (plasma) terhadap waktu (t) dengan konstanta (k) tertentu
def pers_konsentrasi(C, k, t):
    return C - C0 * np.exp(-k * t)

#program komputasi metode Newton-Raphson yang universal dan diubah ke metode Secant
"""definisikan parameter konsentrasi awal (C0), konstanta (k), waktu (t_initial), konsentrasi awal kedua (C_prev),
kriteria pemberhentian (stopping_criteria), dan maksimal pengulangan (iterasi_maks)"""
def newtonRaphson_ke_secant(C0, k, t_initial, C_prev, iterasi_maks = 100, stopping_criteria = 1e-6):
  #definisikan initial guess (konsentrasi) pertama
    C = C0
    for iterasi in range(iterasi_maks):
        f = pers_konsentrasi(C, k, t_initial)
        f_prev = pers_konsentrasi(C_prev, k, t_initial)
        delta_C = f * (C - C_prev) / (f - f_prev)
        #definisikan initial guess (konsentrasi) kedua
        C_prev = C
        C -= delta_C
        if abs(delta_C) < stopping_criteria:
            print(f"\nKonvergen setelah pengulangan ke-{iterasi + 1} dengan metode Secant.")
            return C
    print(f"\nMetode Secant tidak konvergen setelah pengulangan ke-{iterasi_maks}.")
    return None

#program komputasi metode Newton-Raphson untuk dicek konvergensinya
"""definisikan parameter konsentrasi awal (C0), konstanta (k), waktu (t_initial),
kriteria pemberhentian (stopping_criteria), dan maksimal pengulangan (iterasi_maks)"""
def newtonRaphson(C0, k, t_initial, iterasi_maks = 100, stopping_criteria = 1e-6):
    #definisikan initial guess (konsentrasi)
    C = C0
    for iterasi in range(iterasi_maks):
        f = pers_konsentrasi(C, k, t_initial)
        f_prime = -k * C0 * np.exp(-k * t_initial)
        #berikan kondisi untuk menghindari f_prime sama dengan 0
        if abs(f_prime) < 1e-10:
            print("\nIterasi berhenti karena turunan mendekati nol.")
            break
        delta_C = f / f_prime
        C -= delta_C
        if abs(delta_C) < stopping_criteria:
            print(f"\nKonvergen setelah pengulangan ke-{iterasi + 1} dengan metode Newton-Raphson.")
            return C
    print(f"Metode Newton-Raphson tidak konvergen setelah pengulangan ke-{iterasi_maks}.")
    return None

#program komputasi metode Gauss-Seidel untuk dicek konvergensinya
"""definisikan parameter konsentrasi awal (C0), konstanta (k), waktu (t_initial),
kriteria pemberhentian (stopping_criteria), dan maksimal pengulangan (iterasi_maks)"""
def gaussSeidel(C0, k, t_initial, iterasi_maks = 100, stopping_criteria = 1e-6):
    #definisikan initial guess (konsentrasi)
    C = C0
    for iterasi in range(iterasi_maks):
        f = pers_konsentrasi(C, k, t_initial)
        C -= f
        if abs(f) < stopping_criteria:
            print(f"\nKonvergen setelah pengulangan ke-{iterasi + 1} dengan metode Gauss-Seidel.")
            return C
    print(f"\nMetode Gauss-Seidel tidak konvergen setelah pengulangan ke-{iterasi_maks}.")
    return None

#definisi konstanta (k)
k = 0.05

#program meminta input konsentrasi awal (C0), tebakan konsentrasi awal kedua (C_prev), dan waktu (t_initial)
try:
  C0 = float(input("Masukkan terkaan konsentrasi awal (mg/L): "))
  C_prev = float(input("Masukkan terkaan konsentrasi awal ke-2 (mg/L): "))
  t_initial = float(input("Masukkan waktu (jam): "))

  #tampilkan output
  #hasil dengan metode Newton-Raphson
  hasil_newtonRaphson = newtonRaphson(C0, k, t_initial)
  if hasil_newtonRaphson is not None:
      print(f"Konsentrasi obat pada t = {t_initial} jam dengan metode Newton-Raphson: C = {hasil_newtonRaphson:.2f} mg/L")
  #hasil dengan metode Secant
  hasil_secant = newtonRaphson_ke_secant(C0, k, t_initial, C_prev)
  if hasil_secant is not None:
      print(f"Konsentrasi obat pada t = {t_initial} jam dengan metode Secant: C = {hasil_secant:.2f} mg/L")
  #hasil dengan metode Gauss-Seidel
  hasil_gaussSeidel = gaussSeidel(C0, k, t_initial)
  if hasil_gaussSeidel is not None:
      print(f"Konsentrasi obat pada t = {t_initial} jam dengan metode Gauss-Seidel: C = {hasil_gaussSeidel:.2f} mg/L")

except ValueError:
    print("Tolong masukkan input berupa angka numerik.")

In [ ]:
import numpy as np

#fungsi distribusi obat yang akan dihitung
def distribusi_obat(x, parameter):
    k1, k2 = parameter
    return k1 * np.exp(-k2 * x)

#program komputasi metode Newton-Raphson untuk dicek konvergensinya
def newtonRaphson(f, dfdx, x0, stopping_criteria, iterasi_maks):
    x = x0
    for i in range(iterasi_maks):
        dfdx_val = dfdx(x)
        if np.all(np.abs(dfdx_val) < 1e-12):
            print("Metode Newton-Raphson menghasilkan pembagian oleh nol.")
            return None
        x_new = x - f(x) / dfdx_val
        if np.all(np.isnan(x_new)) or not np.all(np.isfinite(x_new)):
            print("Metode Newton-Raphson divergen atau menghasilkan nilai yang tidak terdefinisi.")
            return None
        if np.allclose(x_new, x, rtol = stopping_criteria):
            return x_new
        x = x_new
    print("Metode Newton-Raphson tidak konvergen.")
    return None

#program komputasi metode Newton-Raphson yang universal dan diubah ke metode Secant
def secant(f, x0, x1, stopping_criteria, iterasi_maks):
    for i in range(iterasi_maks):
        f_x0, f_x1 = f(x0), f(x1)
        if np.all(np.isclose(f_x1 - f_x0, 0.0, atol = 1e-12)):
            print("Metode Secant menghasilkan pembagian oleh nol.")
            return None
        x_new = x1 - f_x1 * (x1 - x0) / (f_x1 - f_x0)
        if np.all(np.isnan(x_new)) or not np.all(np.isfinite(x_new)):
            print("Metode Secant divergen atau menghasilkan nilai yang tidak terdefinisi.")
            return None
        if np.allclose(x_new, x1, rtol=stopping_criteria):
            return x_new
        x0, x1 = x1, x_new
    print("Metode Secant tidak konvergen.")
    return None

#program komputasi metode Gauss-Seidel untuk dicek konvergensinya
def gaussSeidel(A, b, x0, stopping_criteria, iterasi_maks):
    n = len(b)
    x = x0.copy()
    for k in range(iterasi_maks):
        for i in range(n):
            sigma = sum(A[i, j] * x[j] for j in range(n) if j != i)
            x[i] = (b[i] - sigma) / A[i, i]
        if np.any(np.isnan(x) | np.isinf(x)):
            print("Metode Gauss-Seidel menghasilkan nilai yang tidak terdefinisi.")
            return None
        if np.allclose(x, x0, rtol = stopping_criteria):
            return x
        x0 = x.copy()
    print("Metode Gauss-Seidel tidak konvergen.")
    return None

#program meminta input jumlah data, waktu, dan konsentrasi plasma
jumlah_data = int(input("Masukkan jumlah data: "))
t_data = np.zeros(jumlah_data)
C_data = np.zeros(jumlah_data)
for i in range(jumlah_data):
    t_data[i] = float(input(f"Waktu data ke-{i + 1}: "))
    C_data[i] = float(input(f"Konsentrasi data ke-{i + 1}: "))

#pemodelan distribusi obat
def model_distribusi_obat(parameter):
    return np.array([distribusi_obat(t, parameter) for t in t_data])

#penyelesaian dengan metode Newton-Raphson
terkaan_param_NR = np.array([1.0, 0.1])
hasil_param_NR = newtonRaphson(lambda params: np.linalg.norm(C_data - model_distribusi_obat(params)),
                                  lambda params: -2 * np.sum((C_data - model_distribusi_obat(params)) * model_distribusi_obat(params)),
                                  terkaan_param_NR, stopping_criteria = 1e-6, iterasi_maks = 100)
if hasil_param_NR is not None:
    print("Solusi Metode Newton-Raphson:", hasil_param_NR)

#penyelesaian dengan metode Secant
terkaan_param_sec = np.array([1.0, 0.1])
hasil_param_sec = secant(lambda params: np.linalg.norm(C_data - model_distribusi_obat(params)),
                          terkaan_param_sec, terkaan_param_NR, stopping_criteria = 1e-6, iterasi_maks = 100)
if hasil_param_sec is not None:
    print("Solusi Metode Secant:", hasil_param_sec)

#penyelesaian dengan metode Gauss-Seidel
A = np.random.rand(jumlah_data, jumlah_data)
b = np.random.rand(jumlah_data)
x0_gaussSeidel = np.zeros_like(b)
solusi_x_gaussSeidel = gaussSeidel(A, b, x0_gaussSeidel, stopping_criteria = 1e-6, iterasi_maks = 100)
if solusi_x_gaussSeidel is not None:
    print("Solusi Metode Gauss-Seidel:", solusi_x_gaussSeidel)